# SyftBox SDK Development Testing

This notebook is for testing and validating the SyftBox SDK during development.

In [1]:
%%capture output --no-stderr

# Install the SDK in development mode (run this once)
!pip install nest_asyncio
!pip install -e .

In [2]:
# Import the SDK
import syft_nsai_sdk as sdk
import nest_asyncio
import httpx
import logging
import os

In [3]:
# import client
from syft_nsai_sdk import SyftBoxClient
from syft_nsai_sdk.utils.logger import get_logger
from IPython.display import display, HTML, Markdown

In [4]:
# define async
nest_asyncio.apply()

# initialize logger
logger = get_logger()
logger.success("Logger initialized successfully!")

# Turn off debug logging for your SDK
logging.getLogger('syft_nsai_sdk').setLevel(logging.INFO)

2025-08-29 23:53:02 - syft_nsai_sdk - SUCCESS - Logger initialized successfully!


In [5]:
# initialize client
client = SyftBoxClient()
logger.info(f"Syftbox client created: {client}")

2025-08-29 23:53:03 - syft_nsai_sdk.main - INFO - Found existing accounting credentials in environment
2025-08-29 23:53:03 - syft_nsai_sdk.main - INFO - SyftBoxClient initialized for guest@syft.org
2025-08-29 23:53:03 - syft_nsai_sdk - INFO - Syftbox client created: <syft_nsai_sdk.main.SyftBoxClient object at 0x1100e35c0>


In [6]:
# # get env vars
# email = os.getenv('SYFTBOX_ACCOUNTING_EMAIL')
# password = os.getenv('SYFTBOX_ACCOUNTING_PASSWORD')
# logger.info(f"SYFTBOX_ACCOUNTING_EMAIL: {email}")
# logger.info(f"SYFTBOX_ACCOUNTING_PASSWORD: {bool(password)}")

# # Setup accounting for paid models
# await client.setup_accounting(email, password)

In [7]:
print(await client.get_account_info())

{'email': 'callis@openmined.org', 'balance': 112.9, 'currency': 'USD', 'account_id': 'cme1ec567000aqn3z9vu7q3st', 'created_at': None}


In [8]:
# Discover models explicitly 
models = client.discover_models(service_type="chat")
print(f"Found {len(models)} models")

2025-08-29 23:53:12 - syft_nsai_sdk.services.health - INFO - Batch health check completed in 5.93s for 7 models
2025-08-29 23:53:12 - syft_nsai_sdk.main - INFO - Discovered 7 models (health_check=True)


Found 7 models


In [9]:
print(f"{'Name':<20} {'Owner':<25} {'Tags':<25} {'Services':<25}")
print("-" * 100)
for model in models:
    enabled_services = [s.type.value for s in model.services if s.enabled]
    tags_str = ", ".join(model.tags[:2])
    services_str = ", ".join(enabled_services)
    print(f"{model.name:<20} {model.owner:<25} {tags_str:<25} {services_str:<25}")

Name                 Owner                     Tags                      Services                 
----------------------------------------------------------------------------------------------------
keelan-test-3        kj@kj.dev                 test                      chat, search             
pal-8                afdolriski@gmail.com                                chat, search             
carl-model           callis@openmined.org      Chat, Search              chat, search             
claude-sonnet-3.5    aggregator@openmined.org  claude, anthropic         chat                     
deepseek-v3          aggregator@openmined.org  deepseek, opensource      chat                     
test                 irina@openmined.org       test                      chat, search             
sota-model           callisezenwaka@barestripe.com chat, search              chat, search             


In [11]:
result = await client.search(
    model_name="carl-model",
    owner="callis@openmined.org", 
    query="Federated learning",
    limit=10,
    similarity_threshold=0.8
)

2025-08-29 23:55:24 - syft_nsai_sdk.main - INFO - Discovered 1 models (health_check=False)


In [12]:
# show response
logger.info(f"Search result:\n\n {result}")

2025-08-29 23:55:34 - syft_nsai_sdk - INFO - Search result:

 Search results for: 'Federated learning'

1. Score: 0.302
   | Private computations                                                                              ...

2. Score: 0.295
   ay to create a digital cash system without requiring a trusted central authority. The idea is to dig...

3. Score: 0.284
   | Alice sends Q A to Bob - -- -- -- -- -- -- -- ---→ Q A                                            ...

4. Score: 0.280
   |
| Further private computations                                                                    ...

5. Score: 0.261
   nd then anyone who knows how to send her email automatically knows her public key. Of course, this i...

6. Score: 0.260
   unctions of Several Variables . Undergraduate Texts in Mathematics, 2nd edn. (Springer, New York, 19...

7. Score: 0.251
   m is to provide security, then their adversary Eve must not know k , must not be able to guess k , a...

8. Score: 0.244
   t use her

In [13]:
# Search multiple models
results = await client.search_multiple_models(
    model_names=["carl-model", "sota-model"],
    query="vacation policy",
    limit_per_model=3,
    total_limit=10,
    remove_duplicates=True
)

2025-08-29 23:55:56 - syft_nsai_sdk.main - INFO - Multi-model search across 2 models
2025-08-29 23:55:56 - syft_nsai_sdk.main - INFO - Discovered 1 models (health_check=False)
2025-08-29 23:55:57 - syft_nsai_sdk.main - INFO - Discovered 1 models (health_check=False)


2025-08-29 23:56:03 - syft_nsai_sdk.main - INFO - Multi-search completed: 3 results from 2/2 models


✓ Response received



In [14]:
# show response
logger.info(f"Search results:\n\n {results}")

2025-08-29 23:56:05 - syft_nsai_sdk - INFO - Search results:

 Search results for: 'vacation policy'

1. Score: 0.195
   are discussed in Sect. 5.3.3.

as illustrated in Fig. 7.6. So our samples are points ( x, y ) in the...

2. Score: 0.189
   the previous i -1 people. Among the 365 possible birthdays, note that the previous i -1 people have ...

3. Score: 0.169
   rations?
- (b) How many hours does it take to perform L (2 250 ) operations?
- (c) How many days doe...
